In [8]:
import sys
import os
import json
import numpy as np
import pandas as pd
from importlib import import_module
import seaborn as sns
import matplotlib.pyplot as plt

# Add the parent directory to the Python path
sys.path.append(os.path.abspath('..'))

from tVAE_base.ctgan_wrapper import TVAESynthesizer
from tVAE_base.local import load_csvs

In [22]:
datasets = load_csvs(folder_name='/home/mfacotti/martin/tVAE_project')

# Access the rhc.csv data using the filename (without extension) as key
rhc_data = datasets['rhc']

# Display basic information about the loaded data
print(f"Dataset shape: {rhc_data.shape}")
rhc_data.head()

Dataset shape: (5735, 62)


/home/mfacotti/martin/tVAE_project/tVAE_base/local.py:36: UserWarning:

Ignoring incompatible files ['EDA.ipynb', 'README.md', '.gitignore', 'numerical_no_na_metadata.json', 'original_metadata.json', 'delete_first_column_rhc.py', 'light_metadata.json', 'ctgan_test.ipynb'] in folder '/home/mfacotti/martin/tVAE_project'.



,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11
